In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet

In [ ]:
df = pd.read_parquet("meow_df.parquet", engine = 'fastparquet')

In [7]:
tmpst = df.query("""time_id<15""")

In [3]:
df.stock_id = df.stock_id.astype(int)

In [4]:
prob = df.query("""stock_id > 126""")

In [5]:
len(prob)

0

In [6]:
len(df)

38382716

In [26]:
df['time_id'].unique()[1]

11

In [2]:
bkdf = pd.read_parquet("optiver/book_train.parquet", engine='fastparquet')

In [11]:
len(bkdf[bkdf['time_id'] == 5].seconds_in_bucket.unique())

600

In [13]:
def price(row):
    return (row['bid_price1'] * row['ask_size1'] + row['ask_price1'] * row['bid_size1']) / (row['bid_size1'] + row['ask_size1'])

In [51]:
bkdf['wap'] = (bkdf['bid_price1'] * bkdf['ask_size1'] + bkdf['ask_price1'] * bkdf['bid_size1']) / (bkdf['bid_size1'] + bkdf['ask_size1'])

In [52]:
bkdf['wap'].describe()

count    1.672533e+08
mean     9.999959e-01
std      3.167179e-01
min      8.830628e-01
25%      9.986876e-01
50%      1.000006e+00
75%      1.001298e+00
max      1.127077e+00
Name: wap, dtype: float64

In [15]:
def volatility(prices):
    log_returns = np.diff(np.log(prices))
    return np.sum(log_returns ** 2)**0.5

In [49]:
def sigma(stock_id, time_id, bkdf):
    prices = bkdf[(bkdf['time_id'] == time_id) & (bkdf['stock_id'] == stock_id)].apply(price, axis = 1).to_numpy()
    return volatility(prices)

In [25]:
print(((bkdf['time_id'] == 5) & (bkdf['stock_id'] == 0)).sum())

302


In [27]:
print(sigma(0, 11, bkdf=bkdf))

200
0.0012044312600105803


In [21]:
train_df = pd.read_csv('optiver/train.csv')

In [29]:
stocks = bkdf['stock_id'].unique().to_list()
print(stocks)

[0, 1, 10, 100, 101, 102, 103, 104, 105, 107, 108, 109, 11, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 122, 123, 124, 125, 126, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 46, 47, 48, 5, 50, 51, 52, 53, 55, 56, 58, 59, 6, 60, 61, 62, 63, 64, 66, 67, 68, 69, 7, 70, 72, 73, 74, 75, 76, 77, 78, 8, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 9, 90, 93, 94, 95, 96, 97, 98, 99]


In [37]:
times = bkdf['time_id'].unique()
print(times)

[    5    11    16 ... 32758 32763 32767]


In [38]:
stocks_df = pd.DataFrame({'stock_id': stocks})
times_df = pd.DataFrame({'time_id': times})
stdf = pd.merge(times_df, stocks_df, how='cross')
print(len(stdf))

428960


In [44]:
before_vs_after = pd.merge(stdf, train_df, how='inner', on=['stock_id', 'time_id'])

In [46]:
before_vs_after.head()

,time_id,stock_id,target
0,5,0,0.004136
1,5,1,0.006340
2,5,10,0.005707
3,5,100,0.004580
4,5,101,0.006088


In [53]:
gb_obj = bkdf.groupby(['stock_id', 'time_id'])

In [ ]:
print(len(gb_obj))

In [50]:
before = before_vs_after.apply(lambda row: sigma(row['stock_id'], row['time_id'], bkdf=bkdf), axis=1)
before_vs_after['before'] = before

KeyboardInterrupt: 